---
## [이용 가이드]

#### 첫번째 셀 : 기본설정 및 크롤링
- 사용자 설정 있음

> `start_page` : 크롤링할 시작 페이지  
> `resresh_unit` : 새로고침 단위 (320 페이지 이상 이동시 크롬 out of memory 발생  
> `stop_page` : 크롤링할 마지막 페이지  
> `file_name` : 저장할 파일명  
- .기호 개당 1초를 의미 <- 사용자 정의 함수 delay(str, int)

#### 두번째 셀 : 데이터 저장

#### 세번째 셀 : 저장된 데이터 확인
- 윈도우에서 인코딩 형식 디스매칭으로 글자 깨질 때 사용하기에 유용
- 인덱스 번호와 게시물 번호 매칭하여 데이터 누락/중복 확인 가능

---

In [2]:
# 첫번째 셀 : 기본설정
# 크롤링을 위해 사용자가 입력해야 되는 값
start_page = 1    # 크롤링할 시작 페이지
refresh_unit = 200   # 크롤링 후 새로고침할 단위
stop_page = 1111   # 크롤링할 마지막 페이지
file_name = 'patent_{date}_{first_page}_{last_page}.csv'  # 저장할 파일명

# 메인
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from IPython.display import Image
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time
import os
import math

# 함수 : 딜레이
def delay(text, sec):
    print(text, end="")
    for i in range(sec): print('.', end=""); time.sleep(1)

# 함수 : 사이트에서 제목/내용 크롤링
current_page_css_selector = 'span.board_pager03 strong'
def contentCrawling(current_page_css_selector = current_page_css_selector):
    dom = BeautifulSoup(driver.page_source, 'html.parser')
    patent_list = dom.select('.search_section article')
    
    current_page = driver.find_element_by_css_selector(current_page_css_selector).text

    result = [
            {
                '제목': patent.select_one('.search_section_title h1 > a:nth-of-type(2)').text.strip(),
                '내용': patent.select_one('.search_txt').text.strip()
            }
            for patent in patent_list
    ]

    return int(current_page), result   # (크롤링한 페이지, 크롤링 결과) 반환

# 함수 : 저장할 파일 경로 찾기
def findPath(file_name):
    file_path = r'D:\DevRoot\StockHelper\dataset'
    file_name = file_name
    try: 
        path = os.path.join(file_path, file_name)
    except(Exception):
        file_path = 'C:\DeepLearning_Project\StockHelper\StockHelper\dataset'
        path = os.path.join(file_path, file_name)
        
    return path

print('프로그램 시작')
start = time.time()  # 시작 시간 저장
result = []

# 1. webdriver를 이용해 kipris 접속
driver_path = r'D:\DevRoot\download\chromedriver.exe'
try: 
    driver = webdriver.Chrome(driver_path)
except(Exception):
    driver_path = r'C:\DevRoot\download\chromedriver.exe'
    driver = webdriver.Chrome(driver_path)

for refreshed_num in range(1, math.ceil(stop_page / refresh_unit) + 1):
    driver.get("http://kpat.kipris.or.kr/kpat/searchLogina.do?next=MainSearch")
    
    # 크롤링 시작페이지 재설정
    if refreshed_num != 1: start_page = refresh_unit * (refreshed_num - 1) + 1
    
    # 2. 검색 옵션 설정
    delay('검색 옵션 설정', 4)

    # 2.1. 행정상태 변경
    # defalut 해제 <- '전체' 클릭
    driver.find_element_by_css_selector('form#leftside .release_list > span:nth-of-type(1) > input').click()
    # 원하는 checkbox만 선택 <- '등록' 클릭
    driver.find_element_by_css_selector('form#leftside .release_list > span:nth-last-of-type(1) > input').click()

    # 2.2. 기간을 검색어로 입력
    today = datetime.today().strftime("%Y%m%d")
    decade = str(int(today) - int('00001000'))
    driver.find_element_by_css_selector('.keyword').send_keys(f'GD=[{decade}~{today}]')
    driver.find_element_by_css_selector('.input_btn img').click()

    # 2.3. 90개씩 보기 선택
    pageSel = 90   # 페이지당 게시물 개수 (30, 60, 90 중 택1)
    select = Select(driver.find_element_by_id('opt28'))
    select.select_by_value(str(pageSel))
    driver.find_element_by_css_selector('#pageSel img').click()

    print('검색 완료')

    # 3. 데이터 추출
    delay('데이터 추출 준비중', 3)
    print('완료')

    page_num = 'span.board_pager03 a:nth-last-of-type({0})'   # target_page 구할 때 이용

    # 3.1. 첫 페이지 크롤링
    current_page, data = contentCrawling()
    if current_page == stop_page: break   # 실행종료
    if current_page != start_page:   # 첫 페이지 찾기
        delay('시작 페이지로 이동', 0)
        while current_page < start_page:
            if current_page // 10 < start_page // 10:
                driver.find_element_by_css_selector(page_num.format(1)).click()
                delay('', 2)
                current_page = int(driver.find_element_by_css_selector(current_page_css_selector).text)
                continue
            for i in range(10, 0, -1):
                driver.find_element_by_css_selector(page_num.format(i)).click()
                delay('', 2)
                current_page = int(driver.find_element_by_css_selector(current_page_css_selector).text)
                if current_page == start_page: print('완료'); break
    current_page, data = contentCrawling()
    result.extend(data)
    print(f'{current_page} 위치 -> {current_page} 페이지 크롤링 완료')

    # 3.2. 페이지 이동하며 크롤링
    while True:
        # 실행 종료
        if current_page >= stop_page: break
        if current_page == refresh_unit * refreshed_num: break

        for i in range(10, 0, -1):
            if current_page >= stop_page: break
            if current_page == refresh_unit * refreshed_num: break

            # 3.2.1. 크롤링할 페이지가 현재 페이지의 다음 페이지인 지 확인
            target_page = driver.find_element_by_css_selector(page_num.format(i))
            if i != 1 and int(target_page.text) <= current_page: delay('', 1); continue 

            print(f'현재 {current_page}', end=" -> ")
            time.sleep(1)
            # 3.2.2. 크롤링할 페이지(target_page)인 다음 페이지로 이동
            print(f'{target_page.text} click', end=" -> ")
            target_page.click()  # 크롤링할 페이지로 이동
            delay('', 2)        
            while True:
                click_page = driver.find_element_by_css_selector(current_page_css_selector).text
    #             try: # (클릭해서 이동한) 현재 페이지(click_page)는 target_page
    #                 click_page = driver.find_element_by_css_selector(current_page_css_selector).text
    #             except KeyboardInterrupt or WebDriverException:
    #                 print('Interrupted')
    #                 try: sys.exit(0)
    #                 except SystemExit: os._exit(0)
    #                 finally:
    #                     print('긁어온 데이터개수 :', len(result))
    #                     print('소요시간 :', int(time.time() - start) / 60, '분')  # 현재시각 - 시작시간 = 실행 시간
    #                     print('프로그램 종료')
    #             except Exception:
    #                 driver.refresh()   # 새로고침
    #                 dealy('', 4)
    #                 dom = BeautifulSoup(driver.page_source, 'html.parser')
    #                 patent_list = dom.select('.search_section article')
    #                 click_page = int(driver.find_element_by_css_selector(current_page_css_selector).text)

                if current_page != click_page: break  # 페이지 이동 시 loop 탈출
                delay("", 2)  # 페이지 이동 안 하면 seelp()
            # 3.2.3. 크롤링
            current_page, data = contentCrawling()
            result.extend(data)
            # 3.2.4. 중간결과 반환
            print(f'{current_page} 페이지 크롤링 완료')

print('=' * 60)
print('긁어온 데이터개수 :', len(result))
print('소요시간 :', int(time.time() - start) / 60, '분')  # 현재시각 - 시작시간 = 실행 시간
print('프로그램 종료')

프로그램 시작
검색 옵션 설정....검색 완료
데이터 추출 준비중...완료
1 위치 -> 1 페이지 크롤링 완료
현재 1 -> 2 click -> ..2 페이지 크롤링 완료
현재 2 -> 3 click -> ..3 페이지 크롤링 완료
현재 3 -> 4 click -> ..4 페이지 크롤링 완료
현재 4 -> 5 click -> ..5 페이지 크롤링 완료
현재 5 -> 6 click -> ..6 페이지 크롤링 완료
현재 6 -> 7 click -> ..7 페이지 크롤링 완료
현재 7 -> 8 click -> ..7 페이지 크롤링 완료
현재 7 -> 9 click -> ..8 페이지 크롤링 완료
현재 8 -> 10 click -> ..10 페이지 크롤링 완료
현재 10 -> 다음 click -> ..11 페이지 크롤링 완료
현재 11 -> 12 click -> ..12 페이지 크롤링 완료
현재 12 -> 13 click -> ..13 페이지 크롤링 완료
현재 13 -> 14 click -> ..14 페이지 크롤링 완료
현재 14 -> 15 click -> ..15 페이지 크롤링 완료
현재 15 -> 16 click -> ..16 페이지 크롤링 완료
현재 16 -> 17 click -> ..17 페이지 크롤링 완료
현재 17 -> 18 click -> ..18 페이지 크롤링 완료
현재 18 -> 19 click -> ..19 페이지 크롤링 완료
현재 19 -> 20 click -> ..20 페이지 크롤링 완료
현재 20 -> 다음 click -> ..21 페이지 크롤링 완료
현재 21 -> 22 click -> ..22 페이지 크롤링 완료
현재 22 -> 23 click -> ..23 페이지 크롤링 완료
현재 23 -> 24 click -> ..24 페이지 크롤링 완료
현재 24 -> 25 click -> ..25 페이지 크롤링 완료
현재 25 -> 26 click -> ..26 페이지 크롤링 완료
현재 26 -> 27 click -> ..27 페이지 크롤링 완료


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=96.0.4664.45)


In [165]:
# 세번째 셀 : 데이터 저장
# 데이터 저장 : list -> df -> csv 저장
file_name = file_name.format(date=today, first_page=1, last_page=len(result) // 90)
path = findPath(file_name)
pd.DataFrame(result, index=range(1, len(result) + 1)).to_csv(path, encoding='utf-8')

# 저장 결과 반환
if os.path.isfile(path):
    print('>>> 파일변환 완료:', datetime.today().strftime(("%Y-%m-%d %H:%M:%S")))
    print('>>> 저장위치:', path)
else: print('>>> 파일변환 실패')

>>> 파일변환 완료: 2021-12-03 09:12:15
>>> 저장위치: D:\DevRoot\StockHelper\dataset\patent_20211203_1_1111.0.csv


In [ ]:
# 네번째 셀 : 저장된 데이터 확인
file_name = file_name.format(date=today)
path = findPath(file_name)
pd.read_csv(path).style.hide_index()